In [33]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re
import string

In [38]:
def cleanSentence(sentence):
    sentence = sentence.split(' ')
    sentence = [ word.strip(string.punctuation + string.whitespace) for word in sentence ]
    sentence = [ word for word in sentence if len(word) > 1 or (word.lower() == 'a' or word.lower() == 'i')]
    return sentence

def cleanInput(content):
    content = content.upper()
    content = re.sub('\n|[[\d+\]]',' ', content)
    content = bytes(content, 'UTF-u')
    content = content.decode('ascii', 'ignore')
    sentences = content.split('. ')
    
    return [cleanSentence(sentence) for sentence in sentences ]

def getNGramsFromSentence(content, n):
    output = []
    for i in range(len(content)-n+1):
        output.append(content[i:i+n])
    return output

def getNGrams(content, n):
    content = cleanInput(content)
    ngrams = []
    for sentence in content:
        ngrams.extend(getNGramsFromSentence(sentence, n))
    return (ngrams)

In [ ]:
html = urlopen('https://en.wikipedia.org/wiki/Python_(programming_language)')
_bs = bs(html, 'html.parser')
content = _bs.find('div', {'id' : 'mw'})